# DCNv2 Sample Walkthrough
이 노트북은 저장소에 포함된 소규모 샘플 데이터를 사용해 DCNv2 모델을 로딩하고 간단히 학습하는 과정을 보여줍니다.

In [ ]:
import sys, pathlib
repo_root = pathlib.Path.cwd().parents[0]
sys.path.append(str(repo_root / 'src'))
from dcnv2_example import DCNv2, load_sample_dataset
import pandas as pdimport tensorflow as tf


In [ ]:
data_path = repo_root / 'data' / 'sample_ranking_data.csv'
df = pd.read_csv(data_path)
df

In [ ]:
dataset = load_sample_dataset(data_path, batch_size=4)
first_batch = next(iter(dataset))
first_batch

In [ ]:
input_dim = first_batch[0].shape[-1]
model = DCNv2(input_dim=input_dim, cross_layers=3, deep_units=(64, 32))
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-3)

for epoch in range(3):
    mean_loss = tf.keras.metrics.Mean()
    for batch_x, batch_y in dataset:
        with tf.GradientTape() as tape:
            preds = model(batch_x, training=True)
            loss = loss_fn(batch_y, preds)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        mean_loss.update_state(loss)
    print(f'Epoch {epoch+1}: loss={mean_loss.result().numpy():.4f}')